In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import numpy as np
import math
from nltk import word_tokenize
import torchtext
from transformers import BertTokenizer, BertModel
from transformers import BertConfig, BertForSequenceClassification

batch_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
train_data = []
train_inp_file = 'processed_data/GCDC/Clinton_train.jsonl'
with open(train_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            json_obj['sentences'][i].append('<EOS>')
        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        train_data.append((document, label-1))
   

test_data = []
test_inp_file = 'processed_data/GCDC/Clinton_test.jsonl'
with open(test_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            json_obj['sentences'][i].append('<EOS>')
        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        test_data.append((document, label-1))
        
print(len(train_data))
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(len(test_data))
print(test_data[0])

800
('two options the us views the transitional national council as the sole / only legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . <eos> this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . <eos> the us views the transitional national council as the legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . <eos> this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . <eos> the inc is the institution through which we are engaging the libyan people at this time . <eos>', 2)
("ambassador , we just received an email from the adoption service provider about these cases . <eos> i am currently reviewing the files 

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert = BertModel.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3)
bert = BertForSequenceClassification.from_pretrained(config)

print(tokenizer.vocab_size)
print(bert.config.hidden_size)

max_pad_len = 300
pad_id = 0

class GCDCDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        document, label = self.data[idx]
        tokenized_document = self.tokenizer.tokenize(document)
        indexed_document = self.tokenizer.convert_tokens_to_ids(tokenized_document)
        indexed_document = indexed_document[:max_pad_len]
        indexed_document = indexed_document + [pad_id] * (max_pad_len - len(indexed_document))
        return torch.tensor(indexed_document), torch.tensor(label)


OSError: Can't load the configuration of 'BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}
'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'BertConfig {
  "architectures": [
    "Be

In [ ]:
    
train_dataset = GCDCDataset(train_data, tokenizer)
test_dataset = GCDCDataset(test_data, tokenizer)

print(len(train_dataset))
print(len(test_dataset))
print(train_dataset[0])
print(test_dataset[0])

800
200
(tensor([ 2048,  7047,  1996,  2149,  5328,  1996, 17459,  2120,  2473,  2004,
         1996,  7082,  1013,  2069, 11476,  6970,  4135, 12690,  2953,  1997,
         1996, 19232,  2111,  2076,  2023,  9455,  2558,  1010,  2004, 19232,
         2015,  2272,  2362,  2000,  2933,  2037,  2219,  2925,  1998,  1037,
         4568,  1010, 18678,  6543,  2291,  2008, 18227,  1996,  2916,  1997,
         2035, 19232,  2015,  1012,  1026,  1041,  2891,  1028,  2023,  2003,
         1999,  5688,  2000,  1996,  1053,  4215,  3270,  8873,  6939,  1010,
         2029,  2038,  2439,  2035, 22568,  2000,  3627,  1012,  1026,  1041,
         2891,  1028,  1996,  2149,  5328,  1996, 17459,  2120,  2473,  2004,
         1996, 11476,  6970,  4135, 12690,  2953,  1997,  1996, 19232,  2111,
         2076,  2023,  9455,  2558,  1010,  2004, 19232,  2015,  2272,  2362,
         2000,  2933,  2037,  2219,  2925,  1998,  1037,  4568,  1010, 18678,
         6543,  2291,  2008, 18227,  1996,  2916,  1997

In [ ]:
# # create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# define a classifier
class Classifier(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.linear = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        return self.linear(x)
    
# define a transformer model
class Transformer(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(Transformer, self).__init__()
        self.bert = bert
        self.classifier = Classifier(hidden_size, num_classes)
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        # output: (batch_size, hidden_size)
        output = self.bert(x)[0][:, 0, :]
        output = self.classifier(output)
        return output

In [ ]:
# define a function to train the model
def train(model, optimizer, criterion, train_loader, test_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for document, label in train_loader:
            # document: (batch_size, seq_len)
            # label: (batch_size)
            document = document.to(device)
            label = label.to(device)
            # output: (batch_size, num_classes)
            output = model(document)
            # output: (batch_size, num_classes)
            loss = criterion(output, label)
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
        # test(model, test_loader)

# define a function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for document, label in test_loader:
            # document: (batch_size, seq_len)
            # label: (batch_size)
            document = document.to(device)
            label = label.to(device)
            # output: (batch_size, num_classes)
            output = model(document)
            # print(output)
            # output: (batch_size)
            output = torch.argmax(output, dim=1)
            correct += torch.sum(torch.eq(output, label)).item()
            total += len(label)
    print('Accuracy: {}'.format(correct / total))

# define a function to predict the label of a document
def predict(model, document):
    model.eval()
    with torch.no_grad():
        # document: (seq_len)
        document = torch.LongTensor(document).unsqueeze(0).to(device)
        # output: (1, num_classes)
        output = model(document)
        # output: (1)
        output = torch.argmax(output, dim=1)
        return output.item()


In [ ]:
# create a transformer model
hidden_size = bert.config.hidden_size
num_classes = 3
model = Transformer(hidden_size, num_classes).to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# train the model
num_epochs = 10
train(model, optimizer, criterion, train_loader, test_loader, num_epochs)
test(model, test_loader)

Epoch: 0, Loss: 1.0586494207382202
Epoch: 1, Loss: 1.041995882987976
Epoch: 2, Loss: 0.9398596286773682
Epoch: 3, Loss: 0.894568920135498
Epoch: 4, Loss: 1.0646461248397827
Epoch: 5, Loss: 1.0431761741638184
Epoch: 6, Loss: 1.2892374992370605
Epoch: 7, Loss: 1.1616665124893188
Epoch: 8, Loss: 1.1563405990600586
Epoch: 9, Loss: 0.7746480703353882
Accuracy: 0.555


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# define a function to evaluate the model
def evaluate(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for document, label in test_loader:
            # document: (batch_size, seq_len)
            # label: (batch_size)
            document = document.to(device)
            label = label.to(device)
            # output: (batch_size, num_classes)
            output = model(document)
            # output: (batch_size)
            output = torch.argmax(output, dim=1)
            y_true.extend(label.tolist())
            y_pred.extend(output.tolist())
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

# evaluate the model
evaluate(model, test_loader)



[[  0   0  51]
 [  0   0  38]
 [  0   0 111]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.00      0.00      0.00        38
           2       0.56      1.00      0.71       111

    accuracy                           0.56       200
   macro avg       0.19      0.33      0.24       200
weighted avg       0.31      0.56      0.40       200



/home/prani/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prani/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prani/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
